In [1]:
from pathlib import Path
from multiprocessing import Pool
import time
from tqdm.notebook import tqdm
from stexls.vscode import *
from stexls.util.workspace import Workspace
from stexls.stex.compiler import Compiler
from stexls.stex.linker import Linker
from stexls.linter.linter import Linter
from stexls.stex.references import *
import os, sys

In [2]:
outdir = Path('/tmp/out')
mathhub = Path('~/MathHub').expanduser()
agenttypes = Path('/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/agent-types.tex')
balancedprime = Path('/home/marian/MathHub/smglom/primes/source/balancedprime.en.tex')
assert mathhub.is_dir()
ws = Workspace(mathhub)
ws.include = '.*/smglom/.*'
print(len(ws.files))
c = Compiler(ws.root, outdir)
ln = Linker(outdir)
global_step = False
linter = Linter(ws, outdir, enable_global_validation=global_step, num_jobs=8)
if global_step:
    it = list(tqdm(linter.compile_workspace()))

2951


In [8]:
ln = linter.lint(agenttypes, on_progress_fun=print)
ln.format_messages()

Preparing 0 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/agent-types.tex 0 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/simple-reflex-agent.tex 1 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/agentenv.tex 2 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/agent-schema.tex 3 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/agent-math.tex 4 False
/home/marian/MathHub/smglom/sets/source/functions.tex 5 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/goal-based-agent.tex 6 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/utility-based-agent.tex 7 False
/home/marian/MathHub/smglom/sets/source/words.tex 8 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/learning-agent.tex 9 False
/home/marian/MathHub/MiKoMH/AI/source/rational-agents/en/stateful-reflex-agent.tex 10 False
/home/marian/MathHub/smglom/sets/source/total-relation.tex 11 False
/home/marian/MathHub/smglom/sets/